## Carbo Loading Live Coding Test

Note: This was done without functions. However, functions could be easily implimented into these answers. Basically, you would tuck the blocks of code into functions with a generalized list being passed in, and then use that function to enter which commodities you want to return. Since we're only working with four specified commodities though, I figured it would be fine to just focus on printing answers.

In [1]:
import pandas as pd

In [2]:
transactions = pd.read_csv('dh_transactions.csv')

In [3]:
store = pd.read_csv('dh_store_lookup.csv')

In [4]:
product = pd.read_csv('dh_product_lookup.csv')

In [5]:
causal = pd.read_csv('dh_causal_lookup.csv')

In [6]:
transactions.head()

,upc,dollar_sales,units,time_of_transaction,geography,week,household,store,basket,day,coupon
0,7680850106,0.80,1,1100,2,1,125434,244,1,1,0
1,3620000470,3.59,1,1100,2,1,125434,244,1,1,0
2,1800028064,2.25,1,1137,2,1,108320,244,2,1,0
3,9999985067,0.85,1,1148,2,1,162016,244,3,1,0
4,9999985131,2.19,1,1323,2,1,89437,244,4,1,0


In [7]:
store.head()

,store,store_zip_code
0,1,37865
1,2,30084
2,3,30039
3,4,31210
4,5,30044


In [8]:
product.head()

,upc,product_description,commodity,brand,product_size
0,111112360,VINCENT S ORIG MARINARA S,pasta sauce,Vincent's,25 OZ
1,566300023,PINE MOUNTAIN SYRUP,syrups,Pine Mountain,40 OZ
2,566300028,MILLER CANE SYRUP,syrups,Miller,19 OZ
3,566300029,MILLER CANE SYRUP,syrups,Miller,12 OZ
4,566300035,PINE MOUNTAIN SYRUP,syrups,Pine Mountain,19 OZ


In [9]:
causal.head()

,upc,store,week,feature_desc,display_desc,geography
0,7680850108,1,68,Wrap Interior Feature,Not on Display,1
1,5100001212,1,66,Wrap Back Feature,Not on Display,1
2,5100002792,1,72,Interior Page Feature,Not on Display,1
3,3620000300,1,55,Wrap Interior Feature,Not on Display,1
4,4112907742,1,68,Wrap Interior Feature,Not on Display,1


## Questions

#### Q1. What are the top five products in each commodity?

In [10]:
# Start by checking what the most common UPCs are in the transactions set. Mode gives us the first.
transactions.upc.mode()

0    9999985020
dtype: int64

In [11]:
# And then value counts gives us the nth most.
upcsales = transactions.upc.value_counts()[:20]

# We can make a new DataFrame of these twenty values.
upcsales = pd.DataFrame(upcsales)
upcsales = upcsales.rename(columns = {'upc':'sales'})
upcsales

,sales
9999985020,110132
9999985004,103971
9999985068,79920
3620000250,79305
9999985021,62020
9999967727,59460
5100002549,53951
3620000300,48643
9999985051,46344
9999985005,45572


In [63]:
# Let's make this a list, and then iterate over the product database. Set index lets us use loc.
# This just gives us the top ten most sold products though.
upclist = transactions.upc.value_counts()[:10].index.tolist()

for i in upclist:
    print(product.set_index('upc').loc[i])

product_description    PRIVATE LABEL THIN SPAGHETTI
commodity                                     pasta
brand                                 Private Label
product_size                                  16 OZ
Name: 9999985020, dtype: object
product_description    PRIVATE LABEL SPAGHETTI REGULAR
commodity                                        pasta
brand                                    Private Label
product_size                                     16 OZ
Name: 9999985004, dtype: object
product_description    PRIVATE LABEL ANGEL HAIR PASTA
commodity                                       pasta
brand                                   Private Label
product_size                                    16 OZ
Name: 9999985068, dtype: object
product_description    RAGU TRADITIONAL PLAIN
commodity                         pasta sauce
brand                                    Ragu
product_size                            26 OZ
Name: 3620000250, dtype: object
product_description    PRIVATE LABEL ELBO MA

In [132]:
# But it's a place to start. So let's make a full count list, a copy of products, and add a column of sales.
product2 = product.copy()
upclist = transactions.upc.value_counts()
upcsales = pd.DataFrame(upclist).reset_index()
upcsales = upcsales.rename(columns = {'index':'upc','upc':'sales'})
upcsales.set_index('upc', inplace=True)
upcsales.loc[9999985020].sales
product2['units_sold'] = 1

i = 0

for group_key, group_value in product2.groupby('upc'):
    product2['units_sold'][i] = upcsales.loc[group_key].sales
    i += 1

<ipython-input-132-d171f8855bf0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product2['units_sold'][i] = upcsales.loc[group_key].sales


In [89]:
# Let's double check that this ran correctly.
product2.set_index('upc').loc[9999985020]

product_description    PRIVATE LABEL THIN SPAGHETTI
commodity                                     pasta
brand                                 Private Label
product_size                                  16 OZ
units_sold                                   110132
Name: 9999985020, dtype: object

In [90]:
product2.set_index('upc').loc[9999985068]

product_description    PRIVATE LABEL ANGEL HAIR PASTA
commodity                                       pasta
brand                                   Private Label
product_size                                    16 OZ
units_sold                                      79920
Name: 9999985068, dtype: object

In [91]:
product2.set_index('upc').loc[9999985021]

product_description    PRIVATE LABEL ELBO MACARONI
commodity                                    pasta
brand                                Private Label
product_size                                 16 OZ
units_sold                                   62020
Name: 9999985021, dtype: object

In [114]:
# Nice! But we have work to do. Let's make a list of all the different commodities to make this nice.
commodities = product2.commodity.unique().tolist()
commodities

['pasta sauce', 'syrups', 'pasta', 'pancake mixes']

In [131]:
# Nice! Now we can set commodity to the index, then use that to pull the top 5 for the commodity list via head()
# Cool trick, we can print nicely in the loop using display() instead of print()

product2.set_index('commodity', inplace=True)

for i in commodities:
    display(product2.loc[i].sort_values(by=['units_sold'], ascending=False).head())

,upc,product_description,brand,product_size,units_sold
commodity,,,,,
pasta sauce,3620000250,RAGU TRADITIONAL PLAIN,Ragu,26 OZ,79305
pasta sauce,5100002549,PREGO REG SPAGHETTI SAUCE,Prego,26 OZ,53951
pasta sauce,3620000300,RAGU OWS SPAG SAUCE MEAT,Ragu,26 OZ,48643
pasta sauce,2700042238,HUNT TRADITIONAL SAUCE,Hunt's,26.5 OZ,45132
pasta sauce,3620000444,RAGU SPAG SCE ITAL GRD CO,Ragu,26.3 OZ,38199


,upc,product_description,brand,product_size,units_sold
commodity,,,,,
syrups,9999967727,PRIVATE LABEL SYRUP PLASTIC BOTTLE,Private Label,24 OZ,59460
syrups,9999966070,PRIVATE LABEL BUTTERED SYRUP,Private Label,P 24 OZ,39254
syrups,3000005970,AUNT JEMIMA ORIGINL SYRUP,Aunt Jemima,24 OZ,31765
syrups,4420979129,MRS BUTTERWORTH SYRUP,Mrs Butterworth,24 OZ,28652
syrups,9999967728,PRIVATE LABEL LITE PANCAKE SYRUP,Private Label,24 OZ,24948


,upc,product_description,brand,product_size,units_sold
commodity,,,,,
pasta,9999985020,PRIVATE LABEL THIN SPAGHETTI,Private Label,16 OZ,110132
pasta,9999985004,PRIVATE LABEL SPAGHETTI REGULAR,Private Label,16 OZ,103971
pasta,9999985068,PRIVATE LABEL ANGEL HAIR PASTA,Private Label,16 OZ,79920
pasta,9999985021,PRIVATE LABEL ELBO MACARONI,Private Label,16 OZ,62020
pasta,9999985051,PRIVATE LABEL SPAGHETTI THIN,Private Label,32OZ,46344


,upc,product_description,brand,product_size,units_sold
commodity,,,,,
pancake mixes,3000005300,AJ BUTTERMILK PANCAKE MIX,Aunt Jemima,32 OZ,43462
pancake mixes,9999985261,PRIVATE LABEL COMPLETE PANCAKE MIX,Private Label,2 LB,31785
pancake mixes,3000005040,AUNT JEM ORIGINAL PANCAKE MIX,Aunt Jemima,2 LB,25542
pancake mixes,3000005070,A/JEM COMPLETE PANCAKE MI,Aunt Jemima,32 OZ,22829
pancake mixes,1800028064,H J PANCK BTRMLK COMP MIX,Hungry Jack,,21504


#### Q2. What are the top 5 brands in each commodity?

In [127]:
# This is similar to Q1, and actually we can use variables from Q1 to solve this.
# We already have the units_sold column, so we use that as a base in product2. Let's reset the index though.

product2.reset_index()

,commodity,upc,product_description,brand,product_size,units_sold
0,pasta sauce,111112360,VINCENT S ORIG MARINARA S,Vincent's,25 OZ,28
1,syrups,566300023,PINE MOUNTAIN SYRUP,Pine Mountain,40 OZ,61
2,syrups,566300028,MILLER CANE SYRUP,Miller,19 OZ,1078
3,syrups,566300029,MILLER CANE SYRUP,Miller,12 OZ,2
4,syrups,566300035,PINE MOUNTAIN SYRUP,Pine Mountain,19 OZ,15
...,...,...,...,...,...,...
922,pasta,9999985217,PRIVATE LABEL ALPHABETS,Private Label,16 OZ,2360
923,pancake mixes,9999985260,PRIVATE LABEL COMPLETE PANCAKE MIX,Private Label,32 OZ,20310
924,pancake mixes,9999985261,PRIVATE LABEL COMPLETE PANCAKE MIX,Private Label,2 LB,31785
925,pasta,9999985488,PRIVATE LABEL ITAL NESTED ANGEL HAIR,Private Label Premium,16 OZ,2306


In [144]:
# First, we build a dictionary for the brands.
brands = dict.fromkeys(product2.brand.unique().tolist(), 0)
brands.items()

dict_items([("Vincent's", 0), ('Pine Mountain', 0), ('Miller', 0), ('Barilla', 0), ("Elena's", 0), ('Boves', 0), ('Fastshake', 0), ('Golden Eagle', 0), ('RR', 0), ('Gooch', 0), ('Alaga', 0), ('M W Flapstax', 0), ('Grandma Molases', 0), ('Dell Amore', 0), ('Creamette', 0), ('Pennsylvania Dutch', 0), ('Healthy Harvest', 0), ('Bisquick', 0), ("Bruce's", 0), ('Hungry Jack', 0), ("Newman's", 0), ('Kraft', 0), ('Brier Rabbit', 0), ('Sobrab Bkstrp', 0), ('Dececco', 0), ('Eden', 0), ('Pomi', 0), ('Annarino', 0), ('La Russa', 0), ("Hunt's", 0), ('Vita', 0), ('Maggi Spaetzle', 0), ("Mother's", 0), ('Mueller', 0), ('La Moderna', 0), ('Aunt Jemima', 0), ('Tree of Life', 0), ('Fifty 50', 0), ('White Lily', 0), ('Sugar Buster', 0), ('Ronzoni', 0), ('San Giorgio', 0), ('China Mandarin', 0), ('Ragu', 0), ('Bertolli', 0), ('Farm Style', 0), ('DaVinci', 0), ('Kellogg', 0), ('Spring Tree', 0), ('Colavita', 0), ('Pasta Shoppe', 0), ('San Marzano', 0), ('Classico', 0), ('B F', 0), ('Krusteaz', 0), ('Pionee

In [146]:
# Now we iterate over the commodities similar to Q1. We're using sum() for this.

for i in commodities:
    comm_temp = product2.loc[i]
    comm_temp.set_index('brand', inplace=True)
   
    # We need to watch out for key errors on this one. And we have to initialize brands.
    brands = dict.fromkeys(product2.brand.unique().tolist(), 0)
    for j in brands:
        try:
            brands[j] += comm_temp.loc[j].units_sold.sum()
        except KeyError:
            continue
        
    x = pd.DataFrame(list(brands.items()),columns = ['brand','units_sold'])
    print(i)
    display(x.sort_values(by=['units_sold'], ascending=False).head())

pasta sauce


,brand,units_sold
43,Ragu,738294
68,Prego,342971
129,Private Label,245385
29,Hunt's,191583
52,Classico,148032


syrups


,brand,units_sold
129,Private Label,206115
35,Aunt Jemima,120901
61,Mrs Butterworth,51197
105,Northwoods,49519
64,Karo,47488


pasta


,brand,units_sold
129,Private Label,998961
3,Barilla,322649
14,Creamette,211008
33,Mueller,203244
128,Private Label Premium,172590


pancake mixes


,brand,units_sold
35,Aunt Jemima,97264
19,Hungry Jack,54633
129,Private Label,52095
54,Krusteaz,13910
38,White Lily,13772


#### Q3. How often is each commodity purchased on average by a customer?

In [13]:
# We're gonna make these next couple questions easier by creating four UPC lists, one for each commodity.
product2 = product.copy()
pasta_upc = product2.set_index('commodity').loc['pasta'].upc.unique().tolist()
psauce_upc = product2.set_index('commodity').loc['pasta sauce'].upc.unique().tolist()
syrup_upc = product2.set_index('commodity').loc['syrups'].upc.unique().tolist()
pancake_upc = product2.set_index('commodity').loc['pancake mixes'].upc.unique().tolist()

In [14]:
# There aren't too many different upcs of pancake mixes, so we can brainstorm ideas with this list.
# Focusing on households is a good starting point. We want to only pull the customers that are actually
# purchasing the commodity for this.

transactions.set_index('upc').loc[pancake_upc]

,dollar_sales,units,time_of_transaction,geography,week,household,store,basket,day,coupon
upc,,,,,,,,,,
1112600710,0.89,1,1227,1,25,299293,200,531796,175,0
1112600710,0.89,1,1820,1,27,299293,200,573271,189,0
1112600710,0.89,1,1639,1,31,256979,200,699184,214,0
1112600710,2.18,2,1439,2,39,80215,313,1224664,273,0
1112600710,3.98,2,1717,1,40,328884,146,1252224,280,0
...,...,...,...,...,...,...,...,...,...,...
9999985261,1.19,1,1900,1,104,288767,158,3315979,726,0
9999985261,1.35,1,2216,1,104,422165,138,3316046,726,0
9999985261,1.35,1,1111,1,104,433035,67,3316073,726,0


In [17]:
# For this subset, we want to look at a couple things. First, how many units at a time are customers
# purchasing? Second, how many units have been sold over this two year period? mean() and sum() are
# good pointers for this.
print('Mean:' + str(transactions.set_index('upc').loc[pancake_upc].units.mean()))
print('Sum:' + str(transactions.set_index('upc').loc[pancake_upc].units.sum()))

Mean:1.112280195488666
Sum:300646


In [27]:
# We're not done with the second part. We need to figure out how many different customers are buying
# pancake mix, and then look at how often that each customer buys the commodity on average.

pm_households = transactions.set_index('upc').loc[pancake_upc].household.value_counts()
pm_households

232513    199
340439     96
254942     69
59972      68
302415     64
         ... 
253714      1
204554      1
227079      1
229126      1
2047        1
Name: household, Length: 130580, dtype: int64

In [28]:
# 130,580 unique customers have bought pancake mix. So, a question arises. How many units in two years does
# someone need to purchase in order to be considered a "customer" for that specific product? Let's set that
# number at more than 1 unit a year for the purpose of this exercise. We do this via where().

pm_households = pm_households.where(pm_households > 1)
pm_households = pm_households.dropna()
pm_households

232513    199.0
340439     96.0
254942     69.0
59972      68.0
302415     64.0
          ...  
25905       2.0
368881      2.0
402972      2.0
423555      2.0
349049      2.0
Name: household, Length: 52130, dtype: float64

In [39]:
# For the sake of this exercise, we are going to use all the data, but the above cell shows how this could be
# done with a condition. (The sum would have to be conditional too to take out the one-time purchases.)

pm_sum = transactions.set_index('upc').loc[pancake_upc].units.sum()
pm_cust = transactions.set_index('upc').loc[pancake_upc].household.value_counts().size

pm_avgunits = pm_sum / pm_cust
print("A customer buys", "%.2f" % pm_avgunits, "units of pancake mix every 2 years.")

A customer buys 2.30 units every 2 years.


In [41]:
# About 2.3 units are sold every two years. What this translates to is:
print("A customer buys", str("%.2f" % (pm_avgunits/2)), "units of pancake mix every year.")
print("A customer buys", str("%.2f" % (pm_avgunits/24)), "units of pancake mix every month.")

A customer buys 1.15 units of pancake mix every year.
A customer buys 0.10 units of pancake mix every month.


In [43]:
# Now we just repeat this for the other three commodities.
# Order will be pasta, syrup, pasta sauce.

p_households = transactions.set_index('upc').loc[pasta_upc].household.value_counts()
s_households = transactions.set_index('upc').loc[syrup_upc].household.value_counts()
ps_households = transactions.set_index('upc').loc[psauce_upc].household.value_counts()

# Pasta calculation.
p_sum = transactions.set_index('upc').loc[pasta_upc].units.sum()
p_cust = transactions.set_index('upc').loc[pasta_upc].household.value_counts().size
p_avgunits = p_sum / p_cust
print("A customer buys", str("%.2f" % (p_avgunits/2)), "boxes of pasta every year.")
print("A customer buys", str("%.2f" % (p_avgunits/24)), "boxes of pasta every month.")

A customer buys 3.50 boxes of pasta every year.
A customer buys 0.29 boxes of pasta every month.


In [44]:
# Syrup calculation.
s_sum = transactions.set_index('upc').loc[syrup_upc].units.sum()
s_cust = transactions.set_index('upc').loc[syrup_upc].household.value_counts().size
s_avgunits = s_sum / s_cust
print("A customer buys", str("%.2f" % (s_avgunits/2)), "units of syrup every year.")
print("A customer buys", str("%.2f" % (s_avgunits/24)), "units of syrup every month.")

A customer buys 1.41 units of syrup every year.
A customer buys 0.12 units of syrup every month.


In [45]:
# Pasta sauce calculation.
ps_sum = transactions.set_index('upc').loc[psauce_upc].units.sum()
ps_cust = transactions.set_index('upc').loc[psauce_upc].household.value_counts().size
ps_avgunits = ps_sum / ps_cust
print("A customer buys", str("%.2f" % (ps_avgunits/2)), "units of pasta sauce every year.")
print("A customer buys", str("%.2f" % (ps_avgunits/24)), "units of pasta every month.")

A customer buys 3.19 units of syrup every year.
A customer buys 0.27 units of syrup every month.


Note that to do this properly, you need to count not how many "units" are purchased, but how many ounces/pounds of the commodity are being purchased. For the sake of time (and keeping the notebook clean, as mass unit conversations and string slicings would have to happen), let's do an example with syrup. Assume the bottles of syrup that sold averaged out to 16oz. Then the "better" answer for that part of the exercise would be:

In [46]:
print("A customer buys", str(1.41 * 16) + "oz of syrup every year.")
print("A customer buys", str(0.12 * 16) + "oz of syrup every month.")

A customer buys 22.56oz of syrup every year.
A customer buys 1.92oz of syrup every month.


#### Q4. How is the performance of the Pasta category? (geography, weekly trends, etc)

#### Q5. In Pasta and Pasta Sauce, what products, if any, are commonly purchased together in the same basket?